In [1]:
import pandas as pd
from keras.models import Sequential, model_from_json
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from joblib import dump

Using TensorFlow backend.


In [2]:
df = pd.read_csv("../data/data_2.csv")
df.drop(['Fighter'], axis=1, inplace=True)
df.T

,0,1,2,3,4,5,6,7,8,9,...,7819,7820,7821,7822,7823,7824,7825,7826,7827,7828
H_odds,140.0,-345.00,-120.00,-420.00,145.00,-155.0,-210.00,-260.00,-420.00,-250.00,...,145.00,100.0,130.000,135.000,-170.000,315.000,127.000,1.500000e+02,315.000,-1.450000e+02
A_odds,-160.0,315.00,100.00,335.00,-165.00,135.0,175.00,220.00,335.00,230.00,...,-155.00,-110.0,-140.000,-145.000,160.000,-345.000,-137.000,-1.600000e+02,-345.000,1.350000e+02
Winner,0.0,1.00,1.00,1.00,0.00,0.0,1.00,1.00,1.00,1.00,...,1.00,1.0,1.000,1.000,1.000,0.000,1.000,0.000000e+00,0.000,0.000000e+00
avg_H_KD,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.00,0.0,0.000,0.500,0.500,0.000,0.000,0.000000e+00,0.000,5.000000e-01
avg_H_SIG_STR_pct,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.00,0.0,0.425,0.640,0.600,0.295,0.630,5.350000e-01,0.505,4.300000e-01
avg_H_TD_pct,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.00,0.0,0.180,0.000,0.165,0.500,0.435,0.000000e+00,0.500,3.300000e-01
avg_H_SUB_ATT,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.00,0.0,0.000,1.000,0.500,0.000,0.500,0.000000e+00,0.000,0.000000e+00
avg_H_PASS,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.00,0.0,0.500,0.000,0.500,0.000,4.500,0.000000e+00,0.500,1.000000e+00
avg_H_SIG_STR_att,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.00,0.0,218.500,40.500,46.000,33.000,76.500,1.300000e+02,108.500,6.950000e+01
avg_H_TOTAL_STR_landed,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.00,0.0,125.000,30.500,30.000,21.500,101.000,7.150000e+01,75.000,4.100000e+01


In [3]:
def ml_pay(row):
    if row['Winner'] == 1:
        if row['H_odds'] < 0:
            pay = 100/(-row['H_odds'])*100
        else:
            pay = row['H_odds']
        return pay
    else:
        return -100
    
final_df = df.copy()
final_df = df.assign(Pay_Out=df.apply(ml_pay, axis=1))
final_df

,H_odds,A_odds,Winner,avg_H_KD,avg_H_SIG_STR_pct,avg_H_TD_pct,avg_H_SUB_ATT,avg_H_PASS,avg_H_SIG_STR_att,avg_H_TOTAL_STR_landed,...,avg_A_GROUND_landed,avg_total_time_fought(seconds),avg_no_of_rounds,total_rounds_fought,Wins,title_bouts,Height_cms,Weight_lbs,age,Pay_Out
0,140.0,-160.0,0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,190.50,185.0,32.0,-100.000000
1,-345.0,315.0,1,0.0,0.000,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,193.04,240.0,34.0,28.985507
2,-120.0,100.0,1,0.0,0.000,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,182.88,185.0,28.0,83.333333
3,-420.0,335.0,1,0.0,0.000,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,170.18,155.0,28.0,23.809524
4,145.0,-165.0,0,0.0,0.000,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,187.96,205.0,29.0,-100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7824,315.0,-345.0,0,0.0,0.295,0.500,0.0,0.0,33.0,21.5,...,1.5,472.0,3.0,4.0,0.0,0.0,175.26,155.0,33.0,-100.000000
7825,127.0,-137.0,1,0.0,0.630,0.435,0.5,4.5,76.5,101.0,...,15.0,900.0,3.0,6.0,1.0,0.0,190.50,205.0,28.0,127.000000
7826,150.0,-160.0,0,0.0,0.535,0.000,0.0,0.0,130.0,71.5,...,0.0,900.0,3.0,6.0,1.0,0.0,172.72,155.0,34.0,-100.000000
7827,315.0,-345.0,0,0.0,0.505,0.500,0.0,0.5,108.5,75.0,...,23.5,900.0,3.0,6.0,1.0,0.0,162.56,115.0,34.0,-100.000000


In [4]:
target = df["Winner"]
# target = final_df["Pay_Out"]
target_names = ["loss", "win"]

In [5]:
data = final_df.drop(columns=['Winner','Pay_Out'],axis=1)
feature_names = data.columns

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=42)

In [7]:
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [8]:
def build_model():
    # create model
    model = Sequential()
    model.add(Dense(units=156, activation='relu', input_dim=52))
    model.add(Dense(units=156, activation='relu'))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=100, activation='relu'))
    model.add(Dense(units=50, activation='relu'))
    model.add(Dense(units=50, activation='relu'))
    model.add(Dense(units=2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
# build
model = build_model()
model.fit(X_train, y_train_categorical, epochs=100, shuffle=True, verbose=2)
model_loss, model_accuracy = model.evaluate(X_test, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
 - 2s - loss: 0.9632 - accuracy: 0.6257
Epoch 2/100
 - 1s - loss: 0.6453 - accuracy: 0.6661
Epoch 3/100
 - 1s - loss: 0.6034 - accuracy: 0.6965
Epoch 4/100
 - 1s - loss: 0.5915 - accuracy: 0.6969
Epoch 5/100
 - 1s - loss: 0.5698 - accuracy: 0.7047
Epoch 6/100
 - 1s - loss: 0.5611 - accuracy: 0.7164
Epoch 7/100
 - 1s - loss: 0.5603 - accuracy: 0.7139
Epoch 8/100
 - 1s - loss: 0.5499 - accuracy: 0.7268
Epoch 9/100
 - 1s - loss: 0.5460 - accuracy: 0.7288
Epoch 10/100
 - 1s - loss: 0.5474 - accuracy: 0.7286
Epoch 11/100
 - 1s - loss: 0.5426 - accuracy: 0.7268
Epoch 12/100
 - 1s - loss: 0.5377 - accuracy: 0.7310
Epoch 13/100
 - 1s - loss: 0.5305 - accuracy: 0.7374
Epoch 14/100
 - 1s - loss: 0.5256 - accuracy: 0.7409
Epoch 15/100
 - 1s - loss: 0.5236 - accuracy: 0.7420
Epoch 16/100
 - 1s - loss: 0.5110 - accuracy: 0.7478
Epoch 17/100
 - 1s - loss: 0.5086 - accuracy: 0.7471
Epoch 18/100
 - 1s - loss: 0.5099 - accuracy: 0.7489
Epoch 19/100
 - 1s - loss: 0.4915 - accuracy: 0.7577
Ep

In [ ]:
#0.787994921207428

In [7]:
param_grid = {
    'n_estimators': [390,400,410],
    'criterion': ['gini','entropy']
}

grid = GridSearchCV(RandomForestClassifier(), param_grid, verbose=True, n_jobs=-1)

grid.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   47.8s finished


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'n_estimators': [390, 400, 410]},
             verbose=True)

In [8]:
print(grid.best_params_)

{'criterion': 'gini', 'n_estimators': 410}


In [9]:
rf = RandomForestClassifier(n_estimators=410,criterion='gini')
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.778203490847169

In [ ]:
# 0.778203490847169

In [10]:
clf = make_pipeline(StandardScaler(),LogisticRegression(max_iter=1000, tol=1e-3))
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(max_iter=1000, tol=0.001))])

In [11]:
clf.score(X_test, y_test)

0.7871434653043848

In [12]:
from joblib import dump, load
dump(clf, 'model.joblib') 

['model.joblib']

In [11]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
regr_1 = DecisionTreeRegressor(max_depth=4)

regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300, random_state=42)

In [12]:
regr_1.fit(X_train, y_train)
regr_2.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=4),
                  n_estimators=300, random_state=42)

In [13]:
regr_1.score(X_test, y_test)

0.37120273926287495

In [14]:
regr_2.score(X_test, y_test)

0.009501168828790263

In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
regr = RandomForestRegressor(max_depth=2, random_state=42)
regr.fit(X_train, y_train)

RandomForestRegressor(max_depth=2, random_state=42)

In [28]:
regr.score(X_test, y_test)

0.284254115905324

In [29]:
from joblib import dump, load
dump(regr, 'payout_model.joblib') 

['payout_model.joblib']

In [17]:
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
regr = BaggingRegressor(base_estimator=SVR(),n_estimators=10, random_state=42)
regr.fit(X_train, y_train)

BaggingRegressor(base_estimator=SVR(), random_state=42)

In [18]:
regr.score(X_test, y_test)

-0.2538134249991155

In [19]:
from sklearn.linear_model import RidgeCV
clf = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]).fit(X_train, y_train)

In [20]:
clf.score(X_test, y_test)

0.27926492673714554

In [21]:
from sklearn.linear_model import LassoCV
reg = LassoCV(cv=5, random_state=0).fit(X_train, y_train)
reg.score(X_test, y_test)

0.27769180804577887

In [22]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svr', SVR(epsilon=0.2))])

In [23]:
regr.score(X_test, y_test)

0.02297738183407827

In [24]:
from sklearn.kernel_ridge import KernelRidge
clf = KernelRidge(alpha=1.0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.27940102010558865

In [25]:
from sklearn import linear_model
clf = linear_model.BayesianRidge().fit(X_train, y_train)
clf.score(X_test, y_test)

0.2793093313114722

In [26]:
from sklearn.linear_model import SGDRegressor
reg = make_pipeline(StandardScaler(),SGDRegressor(max_iter=1000, tol=1e-3)).fit(X_train, y_train)
reg.score(X_test, y_test)

0.270877927484685